## EDA

Lisa Fung, 8/3/24

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import geopandas as gpd

In [15]:
acs_df = pd.read_csv("../Data/acs_5yr_est_selected_economic_characteristics_2010-2022.csv")
commodities_df = pd.read_csv("../Data/all_commodities_fixed.csv")
stocks_prices_df = pd.read_csv("../Data/all_stock_and_etfs.csv")
meat_store_df = pd.read_csv("../Data/Meat_Stats_Cold_Storage.csv")
meat_prod_df = pd.read_csv("../Data/Meat_Stats_Meat_Production.csv")
meat_scounts_df = pd.read_csv("../Data/Meat_Stats_Slaughter_Counts.csv")
meat_sweights_df = pd.read_csv("../Data/Meat_Stats_Slaughter_Weights.csv") # 1983-2023
health_df = pd.read_csv("../Data/Nutrition_Physical_Activity_and_Obesity_Data.csv")
stocks_info_df = pd.read_csv("../Data/stock_descriptions.csv")

C:\Users\lisaf\AppData\Local\Temp\ipykernel_16548\1812283153.py:8: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  health_df = pd.read_csv("../Data/Nutrition_Physical_Activity_and_Obesity_Data.csv")


In [16]:
# commodities_df['Commodity'].unique()
# stocks_prices_df['Ticker_Symbol'].unique()
# meat_store_df['Animal'].unique()
# meat_scounts_df['Animal'].unique()
# stocks_info_df

## Health Data: Nutrition_Physical_Activity_and_Obesity_Data
Questions:
- Ages: Grade 9-12, Adults 18 and older
- Stratification: 'LocationAbbr', 'Age(years)', 'Education', 'Gender', 'Grade', 'Income', 'Race/Ethnicity'
- Questions: Physical Activity, Obesity/Overweight, Nutrition (Vegetables, fruits, soda/pop)

In [70]:
health_df[['QuestionID', 'Question']].value_counts()
# health_df['StratificationCategory1'].value_counts()
# health_df['Stratification1'].value_counts()
# len(health_df['YearStart'] == health_df['YearEnd']) == len(health_df) # YearStart = YearEnd for all

QuestionID  Question                                                                                                                                                                                                                                             
Q036        Percent of adults aged 18 years and older who have obesity                                                                                                                                                                                               18117
Q037        Percent of adults aged 18 years and older who have an overweight classification                                                                                                                                                                          18117
Q047        Percent of adults who engage in no leisure-time physical activity                                                                                                                                   

In [18]:
health_df.columns
# health_df['Age(years)'].value_counts()

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'Class', 'Topic', 'Question', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Alt', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Low_Confidence_Limit', 'High_Confidence_Limit',
       'Sample_Size', 'Total', 'Age(years)', 'Education', 'Gender', 'Grade',
       'Income', 'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID',
       'QuestionID', 'DataValueTypeID', 'LocationID',
       'StratificationCategory1', 'Stratification1',
       'StratificationCategoryId1', 'StratificationID1'],
      dtype='object')

In [19]:
SOCIO_COLS = ['Age(years)', 'Education', 'Gender', 'Grade', 'Income', 'Race/Ethnicity']
# health_df[health_df['Age(years)'].notna()][['Datasource', 'Question', 'LocationAbbr'] + SOCIO_COLS]['LocationAbbr'].value_counts()

In [20]:
health_df.iloc[0]

YearStart                                                                  2019
YearEnd                                                                    2019
LocationAbbr                                                                 AK
LocationDesc                                                             Alaska
Datasource                              Youth Risk Behavior Surveillance System
Class                                                         Physical Activity
Topic                                              Physical Activity - Behavior
Question                      Percent of students in grades 9-12 who achieve...
Data_Value_Unit                                                             NaN
Data_Value_Type                                                           Value
Data_Value                                                                  9.6
Data_Value_Alt                                                              9.6
Data_Value_Footnote_Symbol              

### Health GIS Visualizations

In [83]:
# Obesity, Overweight
# Percent of adults aged 18 years and older who have an overweight classification
health_weight_df = health_df[health_df['ClassID'] == 'OWS'].copy()

In [84]:
health_weight_df.Question.value_counts()

Question
Percent of adults aged 18 years and older who have obesity                         18117
Percent of adults aged 18 years and older who have an overweight classification    18117
Percent of students in grades 9-12 who have obesity                                 5614
Percent of students in grades 9-12 who have an overweight classification            5614
Name: count, dtype: int64

In [90]:
# [['LocationAbbr'] + SOCIO_COLS]
# Q036 = 'Percent of adults aged 18 years and older who have obesity'

health_q36_df = health_weight_df[health_weight_df['QuestionID'] == 'Q036'].copy()
health_q36_df['YearStart'].value_counts() # 2011 - 2022

q36_vars = {'YearStart' : 2022, 
            'StratificationCategory1' : 'Total', 
            'Stratification1' : 'Total'}

q36_mask = (health_q36_df['YearStart'] == q36_vars['YearStart']) & (health_q36_df['StratificationCategory1'] == q36_vars['StratificationCategory1']) & (health_q36_df['Stratification1'] == q36_vars['Stratification1'])

# Heatmap by percentage of obesity by state, for given year, using 'Total'
health_q36_df[q36_mask].info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 48394 to 133279
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   YearStart                   55 non-null     int64  
 1   YearEnd                     55 non-null     int64  
 2   LocationAbbr                55 non-null     object 
 3   LocationDesc                55 non-null     object 
 4   Datasource                  55 non-null     object 
 5   Class                       55 non-null     object 
 6   Topic                       55 non-null     object 
 7   Question                    55 non-null     object 
 8   Data_Value_Unit             0 non-null      float64
 9   Data_Value_Type             55 non-null     object 
 10  Data_Value                  55 non-null     float64
 11  Data_Value_Alt              55 non-null     float64
 12  Data_Value_Footnote_Symbol  0 non-null      object 
 13  Data_Value_Footnote         0 non-